In [1]:
pip install tensorrt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.4 MB/s eta 0:00:00
  Created wheel for tensorrt: filename=tensorrt-10.12.0.36-py2.py3-none-any.whl size=46638 sha256=5968b0b360e605d1700b9737a4f634159b0f326fd967b00f99d5fe43713ebaa6
  Stored in directory: /root/.cache/pip/wheels/15/7e/1d/fa229fa908a941f493812047b942b726d18e66f30fe2ac3854
  Created wheel for tensorrt_cu12: filename=tensorrt_cu12-10.12.0.36-py2.py3-none-any.whl size=17480 sha256=d94c1afb18bd5dc1d972c6f501063d3acefec884eb9217f4f9271648b5345a92
  Stored in directory: /root/.cache/pip/wheels/93/b2/d6/1629763d3e056546381842f6216f736bf45390fc25cf204091
  Created wheel for tensorrt_cu12_libs: filename=tensorrt_cu12_libs-10.12.0.

In [10]:
pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 85.0 MB/s eta 0:00:00


In [2]:
import tensorrt as trt
print(trt.__version__)  # Должно вывести версию TensorRT

10.12.0.36


In [6]:
from torchvision import models

In [11]:
import torch
model = models.resnet18(pretrained=True)
dummy_input = torch.randn(1,3,224,224)  # Пример входа
torch.onnx.export(model, dummy_input, "model.onnx")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
import tensorrt as trt

logger = trt.Logger(trt.Logger.WARNING)
builder = trt.Builder(logger)
network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))
parser = trt.OnnxParser(network, logger)

with open("model.onnx", "rb") as f:
    parser.parse(f.read())

In [2]:
import tensorrt as trt

In [6]:
logger = trt.Logger(trt.Logger.WARNING)

In [ ]:
builder = trt.Builder(logger)

In [1]:
network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))

NameError: name 'builder' is not defined

In [2]:
parser = trt.OnnxParser(network, logger)

with open("model.onnx", "rb") as f:
    parser.parse(f.read())

NameError: name 'trt' is not defined

In [3]:
config = builder.create_builder_config()
config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 1 << 30)  # 1GB
serialized_engine = builder.build_serialized_network(network, config)

# Сохранение Engine
with open("model.engine", "wb") as f:
    f.write(serialized_engine)

NameError: name 'builder' is not defined

In [4]:
runtime = trt.Runtime(logger)
with open("model.engine", "rb") as f:
    engine = runtime.deserialize_cuda_engine(f.read())

NameError: name 'trt' is not defined

In [ ]:
context = engine.create_execution_context()

In [ ]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit

# Выделение памяти на GPU
h_input = np.random.random((1, 3, 224, 224)).astype(np.float32)
d_input = cuda.mem_alloc(h_input.nbytes)
h_output = np.empty((1, 1000), dtype=np.float32)  # Пример выхода
d_output = cuda.mem_alloc(h_output.nbytes)

# Копирование данных на GPU
cuda.memcpy_htod(d_input, h_input)

In [5]:
bindings = [int(d_input), int(d_output)]
context.execute_v2(bindings)

# Копирование результатов обратно
cuda.memcpy_dtoh(h_output, d_output)
print(h_output)

NameError: name 'd_input' is not defined

In [ ]:
config.set_flag(trt.BuilderFlag.FP16)  # Включение FP16

In [ ]:
profile = builder.create_optimization_profile()
profile.set_shape("input_name", min=(1, 3, 224, 224), opt=(1, 3, 224, 224), max=(1, 3, 512, 512))
config.add_optimization_profile(profile)

In [ ]:
pip install torch-tensorrt

In [ ]:
import torch_tensorrt
trt_model = torch_tensorrt.compile(model, inputs=[torch_tensorrt.Input((1, 3, 224, 224))])